# $\rho'$ analysis notes

## GIRD Selection criteria:

These criteria were applied on the selection stage(GRID):

Events:

    * >= 4 tracks
    
Tracks:

    * Has Point On inner or outer ITS Layer
    * Not ITS SA
    * |dca1| < 3 && |dca0| < 3;

### Data info

In [1]:
from modules import events, dfs, np, pd, hep, plt, Pi0, PiPlus, Rho0
from modules.data.info.DataInfo import DataInfo

DataInfo()


name                 | typename                 | interpretation                
---------------------+--------------------------+-------------------------------
RunNum               | int32_t                  | AsDtype('>i4')
PeriodNumber         | uint32_t                 | AsDtype('>u4')
OrbitNumber          | uint32_t                 | AsDtype('>u4')
BunchCrossNumber     | uint16_t                 | AsDtype('>u2')
Mass                 | float                    | AsDtype('>f4')
Pt                   | float                    | AsDtype('>f4')
Q                    | int16_t                  | AsDtype('>i2')
Rapidity             | float                    | AsDtype('>f4')
Phi                  | float                    | AsDtype('>f4')
ZNAenergy            | float                    | AsDtype('>f4')
ZNCenergy            | float                    | AsDtype('>f4')
ZPAenergy            | float                    | AsDtype('>f4')
ZPCenergy            | float                    | AsDtype(

## Analysis criteria

Let's take from input date only events with 4 tracks and zero total charge.
Also prepare mask for tracks that satisfy standatd criteria contains further conditions for the TPC:

* |NumberOfSigmaTPCPion| < 3
* Number of TPC Clusters > 50
* TPCRefit


In [33]:
from modules.data.selection import dfs4Tracks, TPCMask, GetITSnTPCTracksDF, dfs4TracksLowPt, TPCMaskLowPt

## Low energy tracks and TPC

There is an idea about tracks with small energies (low pt) not able to reach TPC.
We can try to estimate influence of TPC criteria to the statistics.

Let's see what happend in case of we will plot pt for any combination of TPC tracks:

### Event with std criteria

|detector|track1|track2|track3|track4|
|:--:|:--:|:--:|:--:|:--:|
|ITS|+|+|+|+|
|TPC|+|+|+|+|

### Let's start to take also such combinations to the sample:

|detector|track1|track2|track3|track4|
|:--:|:--:|:--:|:--:|:--:|
|ITS|+|+|+|+|
|TPC|+|+|+|-|

|detector|track1|track2|track3|track4|
|:--:|:--:|:--:|:--:|:--:|
|ITS|+|+|+|+|
|TPC|+|+|-|-|

|detector|track1|track2|track3|track4|
|:--:|:--:|:--:|:--:|:--:|
|ITS|+|+|+|+|
|TPC|+|-|-|-|

|detector|track1|track2|track3|track4|
|:--:|:--:|:--:|:--:|:--:|
|ITS|+|+|+|+|
|TPC|-|-|-|-|

We can see statistics gain ration equal 2.5 between std criteria and total combinations:


In [34]:
from matplotlib.patches import Rectangle
from modules.physics import kinematics

%matplotlib widget


plt.style.use(hep.style.ROOT)
fig = plt.figure(figsize=(15,7))
ax = fig.add_subplot()

fig.suptitle(f'4track events pt', fontsize=32)
tpcimpPt = []
colors = ['red','green', 'yellow', 'orange', 'black']
labels = ['ITS & (>= 0TPC)','ITS & (>= 1TPC)', 'ITS & (>= 2TPC)', 'ITS & (>= 3TPC)', 'ITS & ( =  4TPC)']

for i in range(5):
    ITSnTPCEvents = pd.unique(dfs4Tracks.reset_index().entry)[TPCMask.groupby('entry').sum() >= i]
    ITSnTPCTracks = dfs4Tracks.loc[ITSnTPCEvents]
    tmpPt = kinematics(ITSnTPCTracks).EventsVectors.pt
    tpcimpPt.append(tmpPt)
    ax.hist(tpcimpPt[i], bins=100, range=(0,2), histtype='step', color=colors[i], linewidth=2, label=labels[i]+f';{(tmpPt<0.15).sum()}')

ax.set_xlabel('$p_t, GeV$')
ax.set_ylabel('# events')

ax.add_patch(Rectangle((0,0.15),0.15,900,fc='lightgrey',alpha=0.4))

ax.legend()


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Now  let's try to see what tracks we lost from signal area and what contribution they have:

In [35]:
from modules.physics.analysis.ITSvsTPC import *

%matplotlib widget

ShowComparison('$p_t$',[AllTPCTracksPt, ITSDiffTPCTracksPt, TotalLowPt], 'GeV', ['ITS&&TPC', 'OnlyITSFromNTPCEvents', 'ITS||TPC'])




Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

TPC and ITS has different coverage for polar angle:

![img1](https://camo.githubusercontent.com/9a7ab40d0f74a866a7095986644134a0f72cc58b/68747470733a2f2f70702e757365726170692e636f6d2f633835323231362f763835323231363738332f3131396137332f304f76685f6c544b4e7a552e6a7067)

Perhaps we have tracks that not only can't reach TPC, but also has $\theta$ values that TPC doesn't cover.

Below we can see polar angle distribution for tracks that covers three cases:

1. All tracks from events were reconstructed by ITS and TPC
2. Only ITS tracks from events with only part TPC tracks. Here tracks that not reconstructed by TPC
3. All tracks from events were reconstructed by ITS or TPC

We can see small gaps with for the second case, that allow to speak about correctness of the suggestion, but anyway low energy of tracks is the main reason why TPC can't reconstructed tracks.  

In [36]:
ShowComparison('$\\theta$',[AllTPCTracksTheta, ITSDiffTPCTracksTheta, TotalLowPtTheta], '$^\\circ$', ['ITS&&TPC', 'OnlyITSFromNTPCEvents', 'ITS||TPC'], nBins=80, ranges=(25,150))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Mass

In our process 4 pions were producted. The most probably intermediate state including two pions and $\rho$ i.e.
$$\rho' \rightarrow \rho \ \pi^+ \pi^- \rightarrow \pi^+ \pi^- \pi^+ \pi^-$$ 

We can see this on distribution of mass that can be obtained as all combinations of pairs from intial four tracks, i.e. only four pairs:

![img](https://camo.githubusercontent.com/53a52e2a6d4ae7112d74f4073c979a51166170d8/68747470733a2f2f70702e757365726170692e636f6d2f633835333632342f763835333632343436372f34383466332f5431375a754b597062526f2e6a7067)

One of these pair should rely to $\rho$

In [99]:
from modules.physics.analysis.mass import LiteHeavyRecoil, LiteHeavyTotal

%matplotlib widget

# TODO add bar errors 
plt.style.use(hep.style.ROOT)
fig, ax = plt.subplots(1,2,figsize=(15,7))
fig.suptitle('Lightest and Recoil Pairs', fontsize=32)
_ = ax[0].hist(LiteHeavyRecoil.Lite,bins=100, range=(0,2), histtype='step', color='blue', linewidth=2, label='lite pair')
_ = ax[0].hist(LiteHeavyRecoil.Recoil,bins=100, range=(0,2), histtype='step', color='red', linewidth=2, label='rest pair')
ax[0].set_xlabel('$Mass, GeV$')
ax[0].set_ylabel('# events')
_ = ax[1].hist2d(LiteHeavyRecoil.Recoil, LiteHeavyRecoil.Lite, bins=(50, 50),range=[(0,2),(0,2)], cmap=plt.cm.jet)
ax[1].set_ylabel('Lightest pair Mass, GeV')
ax[1].set_xlabel('Recoiling pair Mass, GeV')
ax[1].legend()


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

No handles with labels found to put in legend.


In [100]:

%matplotlib widget

# TODO add bar errors 
plt.style.use(hep.style.ROOT)
fig, ax = plt.subplots(1,2,figsize=(15,7))
fig.suptitle('All Pairs', fontsize=32)

_ = ax[0].hist(LiteHeavyTotal.Lite,bins=100, range=(0,2), histtype='step', color='blue', linewidth=2, label='lite pair')
_ = ax[0].hist(LiteHeavyTotal.Recoil,bins=100, range=(0,2), histtype='step', color='red', linewidth=2, label='rest pair')
ax[0].set_xlabel('$Mass, GeV$')
ax[0].set_ylabel('# events')
_ = ax[1].hist2d(LiteHeavyTotal.Recoil, LiteHeavyTotal.Lite, bins=(50, 50),range=[(0,2),(0,2)], cmap=plt.cm.jet)
ax[1].set_ylabel('Lightest pair Mass, GeV')
ax[1].set_xlabel('Recoiling pair Mass, GeV')
ax[1].legend()


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

No handles with labels found to put in legend.


## Debug

Something wrong with my mass distribution. It has too wide peak for recoiling pairs. Let's see what other parameters have tracks that rely for this bias.

In [13]:
c = (0.4 <= secondCombSorted) * (secondCombSorted <= 0.6)
clist = c.index[(c.Lite * c.Heavy)].to_list()
print(clist[:10])
ITSnTPCTracks.loc[521]

[521, 600, 807, 921, 1168, 1356, 1491, 1695, 1930, 2160]


,T_NumberOfSigmaTPCPion,T_Eta,T_Phi,T_Px,T_Py,T_Pz,T_Q,T_TPCNCls,T_TPCRefit
subentry,,,,,,,,,
0,0.796625,0.519204,4.893516,0.074929,-0.409151,0.225801,1.0,147.0,1.0
1,-0.277492,0.489071,2.217984,-0.376581,0.498271,0.317782,-1.0,143.0,1.0
2,-1.512271,0.492116,1.338488,0.071980,0.304254,0.160147,1.0,140.0,1.0
3,-0.179922,0.233628,6.112863,0.246390,-0.042376,0.058941,-1.0,58.0,1.0


In [26]:
# get first possible combinations of pairs
# FirstPair  = 
# (PosFirstTrack.T_Eta - NegFirstTrack.T_Eta)
# firstCombSorted.index.to_list()
ITSnTPCTracks.loc[26]
# SecPairMass    = GetPairMass(PosSecTrack,   NegSecTrack)

# # get second possible combinations of pairs
# ThirdPairMass  = GetPairMass(PosFirstTrack, NegSecTrack)
# FourthPairMass = GetPairMass(PosSecTrack,   NegFirstTrack)

,T_NumberOfSigmaTPCPion,T_Eta,T_Phi,T_Px,T_Py,T_Pz,T_Q,T_TPCNCls,T_TPCRefit
subentry,,,,,,,,,
0,0.117312,0.084907,5.399884,1.001586,-1.219759,0.134169,-1.0,158.0,1.0
1,-0.190391,0.573115,4.870948,0.103258,-0.645757,0.395652,1.0,156.0,1.0
2,-999.000000,0.117570,5.857540,0.107440,-0.048709,0.013901,-1.0,0.0,0.0
3,-999.000000,0.875668,0.795731,0.128050,0.130724,0.181517,1.0,0.0,0.0


In [ ]:
# get first possible combinations of pairs
FirstPairMass  = GetPairMass(PosFirstTrack, NegFirstTrack)
SecPairMass    = GetPairMass(PosSecTrack,   NegSecTrack)

# get second possible combinations of pairs
ThirdPairMass  = GetPairMass(PosFirstTrack, NegSecTrack)
FourthPairMass = GetPairMass(PosSecTrack,   NegFirstTrack)

## Runs stats

Let's see what run have maximum events for our data when TPC track is 'n':

In [12]:
import collections
runs = events.arrays(filter_name=['RunNum'],library='np')
collections.Counter(runs['RunNum'][np.unique(ITSnTPCTracks.reset_index().entry.to_numpy())]).most_common()[:10]


[(246087, 725),
 (246989, 573),
 (246980, 555),
 (245507, 458),
 (246984, 414),
 (245963, 408),
 (245407, 391),
 (245554, 387),
 (245775, 361),
 (246488, 338)]